In [ ]:
#DEV
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import tables as tb
import h5py
import scipy.stats
import datetime
from copy import deepcopy
from IPython.display import clear_output, display, HTML, Image
import matplotlib.cm as cm
import warnings

warnings.filterwarnings("ignore")
import sys
import pickle
import matplotlib.pyplot as plt

if OS() == 'Linux':
    root = "/data"
elif OS() == 'Windows':
    root = "C:\\DATA\\"
else:
    root = "/Users/davidrobbe/Documents/Data/"

param={
    "goalTime":7,#needed for pavel data only
    "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
    "maxTrialDuration":15,
    "interTrialDuration":10,#None pavel
    "endTrial_frontPos":30,
    "endTrial_backPos":55, 
    "endTrial_minTimeSec":4,
    "cameraSamplingRate":25, #needed for new setup    

    "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
    "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
    "nbJumpMax":100,#200 pavel
    "binSize":0.25,
    #parameters used to preprocess (will override the default parameters)
}  
ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
CWD=os.getcwd()
os.chdir(CommonNoteBookesPath)
%run Animal_Tags.ipynb
%run UtilityTools.ipynb
%run BatchRatBehavior.ipynb
%run plotRat_documentation_3_KinematicsInvestigation.ipynb
%run plotRat_documentation_1_GeneralBehavior.ipynb
%run loadRat_documentation.ipynb
%run RunBatchRat_3_CompareGroups.ipynb

os.chdir(CWD)

In [ ]:
get_pattern_animalList(root,'Late*DMS*')

# Saving data for Stef

In [ ]:
profilePost={'Type': 'Good',
 'rewardType': 'Progressive',
 'option': ['not used', 'AsymmetricLesion'],
 'initialSpeed': ['0', '10'],
 'Speed': '10',
 'Tag': ['Late-Lesion_DLS',
  'Late-Lesion_DS',
  'Late-Lesion_DMS',
  'Late-Lesion_DS-Sharp',
  'Late-Lesion_DMS-Sharp']}

profilePre={'Type':'Good',
          'rewardType':'Progressive',
          'initialSpeed':['10','0'],
          'Speed':'10',
          'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control',
                 'Control-Sharp','IncReward-Late-Sharp','Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
          }

preSlice =slice(-5,None)
postSlice=slice(0,3)
finSlice =slice(8,13)
Param="Forward Running Speed"
Param="median entrance time (sec)"
Param="Maximum Position"
Param="standard deviation of entrance time"

animalList=batch_get_animal_list(root, profilePost)

In [ ]:
len(animalList)

In [ ]:
Before=dict.fromkeys(animalList,list())
After=dict.fromkeys(animalList,list())
Final=dict.fromkeys(animalList,list())

for animal in animalList:
    #Before Lesion
    Before[animal]=animal_learning_stats(root, animal, PerfParam=Param, profile=profilePre,
                          sessionSlice=preSlice)
    
    #After Lesion
    After[animal]=animal_learning_stats(root, animal, PerfParam=Param, profile=profilePost,
                          sessionSlice=postSlice)
    
    #Final Lesion
    Final[animal]=animal_learning_stats(root, animal, PerfParam=Param, profile=profilePost,
                          sessionSlice=finSlice)

    

with open(os.path.join(root,'Stef','Before',f'{Param}.p'),'wb') as f:
    pickle.dump(Before, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(root,'Stef','After',f'{Param}.p'),'wb') as f:
    pickle.dump(After, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(os.path.join(root,'Stef','Final',f'{Param}.p'),'wb') as f:
    pickle.dump(Final, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#Define other measures
def _sd_speed(data):
    speed=forwardRunningSpeed(data).compute()
    result=np.std(list(speed.values()))
    return result

def _sd_max_pos(data):
    seqObj=sequentialTrials(data)
    *_,peaks=seqObj._compute_max_pos()
    if len(peaks) > 10:
        maxPos=np.nanstd(peaks)
    else:
        maxPos=np.nan

    return maxPos

def _max_pos_traj(data):
    seqObj=sequentialTrials(data)
    _,trials,_=seqObj._compute_max_pos()
    res=[data.position[t] for t in trials]
    return res


#run the following line by line
Param=_sd_speed.__name__
Param=_sd_max_pos.__name__
Param=_max_pos_traj.__name__

In [ ]:
Before=dict.fromkeys(animalList,list())
After=dict.fromkeys(animalList,list())
Final=dict.fromkeys(animalList,list())

for animal in animalList:
    #Before Lesion
    Before[animal]=animal_learning_stats(root, animal, PerfParam=Param, profile=profilePre,
                          sessionSlice=preSlice,redo=False)
    
    #After Lesion
    After[animal]=animal_learning_stats(root, animal, PerfParam=Param, profile=profilePost,
                          sessionSlice=postSlice,redo=False)
    
    #Final Lesion
    Final[animal]=animal_learning_stats(root, animal, PerfParam=Param, profile=profilePost,
                          sessionSlice=finSlice,redo=False)

    

with open(os.path.join(root,'Stef','Before',f'{Param}.p'),'wb') as f:
    pickle.dump(Before, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(root,'Stef','After',f'{Param}.p'),'wb') as f:
    pickle.dump(After, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(os.path.join(root,'Stef','Final',f'{Param}.p'),'wb') as f:
    pickle.dump(Final, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
if "__file__" not in dir():
    profileLesion={'Type': 'Good',
     'rewardType': 'Progressive',
     'Speed': '10',
     'Tag': ['Late-Lesion_DLS','Late-Lesion_DS-Sharp','Late-Lesion_DS','Late-Lesion_DMS','Late-Lesion_DMS-Sharp',
            'Control','Control-BackTo10','Control-Late-BackTo10','Control-AfterBreak','Control-Sharp',
             'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control','IncReward-Late-Sharp',
             'Control-Sharp-AfterBreak','ImmobileTreadmill-Control',
            'Early-Lesion_DLS','Early-Lesion_DMS','Early-Lesion_DMS-NoHab','Early-Lesion_DS','Early-Lesion_DS-NoHab',
            'Reverse','Reverse-AfterBreak','Reverse-Late_DLS']
                  }

    Params=["Forward Running Speed",'median entrance time (sec)',"Motor Sequence Score","Maximum Position",
           "standard deviation of entrance time",]
    
    animalList=batch_get_animal_list(root,profileLesion)
    
    for animal in animalList:
        for perfParam in Params:
            
            animal_learning_stats(root, animal, PerfParam=perfParam, profile={'Type':'Good'}, redo=False)

In [ ]:
import PyPDF2 as ppdf

In [ ]:
file1='/home/david/Documents/TEMP/Task_Example_Group.pdf'
file2='/home/david/Documents/TEMP/TreadmillSketch.pdf'
file3='/home/david/Documents/TEMP/NEW.pdf'

In [ ]:
f1=ppdf.PdfFileReader(file1).getPage(0)
f2=ppdf.PdfFileReader(file2).getPage(0)

f1.mergeTranslatedPage(page2=f2, tx=55, ty=245, expand=False)

writer=ppdf.PdfFileWriter()
writer.addPage(f1)
with open(file3,'wb') as f3:
    writer.write(f3)

# Max Pos Comparison

In [ ]:
profile={'Type':'Good',
         'rewardType':'Progressive',
         'initialSpeed':['0','10'],
         'Speed':'10',
         'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout',
          'Control-NoTimeout-Control','Control-Sharp','IncReward-Late-Sharp',
          'Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
         }
animalList=batch_get_animal_list(root,profile)

profile2={'Type':'Good',
     'option': ['not used','AsymmetricLesion'],
     'rewardType':'Progressive',
     'initialSpeed':['10','0'],
     'Speed':'10',
     'Tag':['Early-Lesion_DS','Early-Lesion_DS-NoHab','Early-Lesion_DMS','Early-Lesion_DLS',
           'Late-Lesion_DMS','Late-Lesion_DLS','Late-Lesion_DS',
           'Late-Lesion_DMS-Sharp','Late-Lesion_DLS-Sharp','Late-Lesion_DS-Sharp']
              }
animalList2=batch_get_animal_list(root,profile2)
PerfParam=["Maximum Position"]
sessionSlice=slice(-5,None)

In [ ]:
dataCtrl=[]
for animal in animalList:
    dataCtrl.append(animal_learning_stats(root, animal, PerfParam[0], profile, sessionSlice))
    
dataCtrl=np.array(dataCtrl)
dataCtrl=np.nanmedian(dataCtrl,axis=1)
dataCtrl=dataCtrl[np.logical_not(np.isnan(dataCtrl))]

dataLes=[]
for animal in animalList2:
    dataLes.append(animal_learning_stats(root, animal, PerfParam[0], profile2, sessionSlice))
    
dataLes=np.array(dataLes)
dataLes=np.nanmedian(dataLes,axis=1)
dataLes=dataLes[np.logical_not(np.isnan(dataLes))]

In [ ]:
plt.boxplot((dataCtrl,dataLes))
plt.ylim([0,90])
plt.show()

In [ ]:
np.nanpercentile(bootstrap(dataCtrl,10000),[1,99]),np.nanpercentile(bootstrap(dataLes,10000),[1,99])

# TRIANGLE

In [ ]:
session='Rat249_2018_08_21_13_02'
data=Data(root,session[:6],session,param={},redoPreprocess=False, saveAsPickle=False);
pos=data.position[30]

In [ ]:
data.maxTrialDuration


In [ ]:
class points:
    def __init__(self,x,y):
        self.x=x
        self.y=y

class edges:
    def __init__(self,A: points, B: points, traj):
        line1=traj[A.x:B.x]
        line2=traj[B.x:C.x]
        time1=np.arange(len(line1))
        time2=np.arange(len(line2))
        
        coef, residuals, *_=np.polynomial.polynomial.polyfit(time1, line1, deg=1, full=True)
        
    
class tri:
    
    """
    make sure trial position exists and entrance time is not NaN
    """
    def __init__(self, data, trial):
        self.trial=trial
        self.traj=data.position[trial]
        self.cs=data.cameraSamplingRate
        self.delay=int(data.cameraToTreadmillDelay*self.cs)
        self.startIndex=self.cs*self.delay
        self.maxTrialDuration=int(data.maxTrialDuration[3]*self.cs)
        
        self.A=self.B=self.C=self.et=None
        
        self._read_et(data)
        self.make_points()
        self.make_edeges()
        
        
    def make_points(self):
        self.A= points(startIndex,self.traj[startIndex])
        
        self.B= points(np.argmax(self.traj[startIndex+1:self.et]) , np.amax(self.traj[startIndex+1:self.et]))
        
        self.C= points(self.et,self.traj[self.et])
        
    def _read_et(self,data):
        try:
            self.et=int(data.entranceTime[self.trial]*self.cs)+self.delay
        except:
            self.et=np.nan
        
        if abs(self.et-self.maxTrialDuration)<10:
            self.et=len(self.traj)-1


# Are first trials faster?

In [ ]:
profile={'Type':'Good',
         'rewardType':'Progressive',
         'initialSpeed':['0','10'],
         'Speed':'10',
         'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout',
          'Control-NoTimeout-Control','Control-Sharp','IncReward-Late-Sharp',
          'Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
         }
animalList=batch_get_animal_list(root,profile)
PerfParam=["Forward Running Speed on Last 40","Forward Running Speed on First 40"]
sessionSlice=slice(-5,None)

In [ ]:
dataLast=[]
dataFrst=[]
for animal in animalList:
    dataLast.append(animal_learning_stats(root, animal, PerfParam[0], profile, sessionSlice))
    dataFrst.append(animal_learning_stats(root, animal, PerfParam[1], profile, sessionSlice))
    
dataLast=np.array(dataLast)
dataFrst=np.array(dataFrst)

In [ ]:
plt.hist(np.nanmean(dataLast-dataFrst,axis=1),20);
plt.xlim([-25,25]);

lesioned animals

In [ ]:
profile2={'Type':'Good',
     'option': ['not used','AsymmetricLesion'],
     'rewardType':'Progressive',
     'initialSpeed':['10','0'],
     'Speed':'10',
     'Tag':['Early-Lesion_DS','Early-Lesion_DS-NoHab','Early-Lesion_DMS','Early-Lesion_DLS',
           'Late-Lesion_DMS','Late-Lesion_DLS','Late-Lesion_DS',
           'Late-Lesion_DMS-Sharp','Late-Lesion_DLS-Sharp','Late-Lesion_DS-Sharp']
              }
animalList2=batch_get_animal_list(root,profile2)
PerfParam=["Forward Running Speed on Last 40","Forward Running Speed on First 40"]
sessionSlice=slice(-5,None)

In [ ]:
dataLast2=[]
dataFrst2=[]
for animal in animalList2:
    dataLast2.append(animal_learning_stats(root, animal, PerfParam[0], profile2, sessionSlice))
    dataFrst2.append(animal_learning_stats(root, animal, PerfParam[1], profile2, sessionSlice))
    
dataLast2=np.array(dataLast2)
dataFrst2=np.array(dataFrst2)

In [ ]:
plt.hist(np.nanmean(dataLast2-dataFrst2,axis=1),20);
plt.xlim([-25,25]);

In [ ]:
test=permtest(np.nanmean(dataLast-dataFrst,axis=1),
              np.nanmean(dataLast2-dataFrst2,axis=1),
              10000)
test.pVal

In [ ]:
CI1=bootstrap(np.nanmean(dataLast-dataFrst,axis=1),10000);
CI2=bootstrap(np.nanmean(dataLast2-dataFrst2,axis=1),10000);

print(np.nanpercentile(CI1,[5,95]),np.nanpercentile(CI2,[5,95]))

In [ ]:
plt.boxplot(x=[np.nanmean(dataFrst,axis=1)[1:],np.nanmean(dataLast,axis=1)],whis=[5,95],
                      positions=[0,1]);
plt.ylim([20,100]);

In [ ]:
plt.boxplot(x=[np.nanmean(dataFrst2,axis=1)[1:],np.nanmean(dataLast2,axis=1)],whis=[5,95],
                      positions=[0,1]);
plt.ylim([20,100]);

#### ViewPoint intertrial analysis for Loco test

check normal exploration

In [ ]:
def find_persistent_movement_epochs(pos: np.array, TH: int, treadmillStep: float =0):
    """
    this function finds epochs in the position of a single trial, pos, where the animal has moved
    for a duration of at least, TH*cameraSamplingRate
    pos: position as a np.array
    TH: an integer like: TH=int(0.2*data.cameraSamplingRate)
    treadmillStep: a float number to correct for treadmill movement, like:
        treadmillStep=np.expand_dims(data.treadmillSpeed[trial]/data.cameraSamplingRate,1)
    """
    
    dp=np.append(np.diff(pos)-treadmillStep,0)
    dp=dp>=0
    dp=dp*1
    dpcross=np.absolute( np.append(np.diff(dp),0)-np.append(0,np.diff(dp)) )
    #any 1 in dpcross mean a zero-crossing in diff(pos)
    crossindex=np.where(dpcross)[0]
    movementepoch=np.where(np.diff(crossindex)>TH)[0]

    posMoveIndex=np.empty(shape=(len(movementepoch),2))

    for i,index in enumerate(movementepoch):
        posMoveIndex[i,:]=(crossindex[index],crossindex[index+1])
        
    return posMoveIndex

def run_distance(data,trial=None):
    if trial is None:
        trial=list(data.position.keys())
    elif not hasattr(trial,'__iter__'):
        trial=[trial]
        
    dis=[0]
    for i in trial:
        et=(data.entranceTime[i]+data.cameraToTreadmillDelay)*data.cameraSamplingRate
        allTraj=data.position[i]
    #     allTraj=scipy.ndimage.filters.gaussian_filter1d(allTraj, sigma=data.cameraSamplingRate/5, truncate=3.0)
        if np.isnan(et):
            th=(data.maxTrialDuration[i]+data.cameraToTreadmillDelay)*data.cameraSamplingRate
            th=int(th)
        else:
            th=int(et)
        allTraj[th-1:]=np.nan
        treadmillStep=np.expand_dims(data.treadmillSpeed[i]/data.cameraSamplingRate,1)
        posMoveIndex=find_persistent_movement_epochs(allTraj,
                                                     TH=int(0.6*data.cameraSamplingRate),
                                                     treadmillStep=treadmillStep)

        for i,j in posMoveIndex:
            epochPos=allTraj[int(i):int(j)]
            dis[-1]+=np.nansum(np.absolute(np.diff(epochPos)-treadmillStep))
        dis.append(0)

    return np.array(dis[:-1])

def intertrial_displacement(data,trial=None):
    if trial is None:
        trial=list(data.position.keys())
    elif not hasattr(trial,'__iter__'):
        trial=[trial]
    
    dis=[0]
    for i in trial:
        if data.dataType != 'ViewPoint':
            allTraj= data.position[i][int((-data.interTrialDuration[i]+1)*data.cameraSamplingRate):]
            TH=int(0.6*data.cameraSamplingRate)
        else:
            allTraj=data.rawIntertrialPosition[i]
            TH=int(0.6*data.cameraSamplingRateInterTrial)

        posMoveIndex=find_persistent_movement_epochs(allTraj,
                                                     TH=TH,
                                                     )

        for i,j in posMoveIndex:
            epochPos=allTraj[int(i):int(j)]
            dis[-1]+=np.nansum(np.absolute(np.diff(epochPos)))
        dis.append(0)

    return np.array(dis[:-1])

In [ ]:
animalList=['Rat325', 'Rat337', 'Rat341', 'Rat345']#, 'Rat329', 'Rat333']
profile={'Type':'Good',
         'rewardType':'Progressive',
#          'initialSpeed':'10',
#          'Speed':['10'],
         'Tag':['Reverse-Late_DLS-EXP','Early-Lesion_DS']
         }
sessionList=batch_get_session_list(root,animalList,profile)['Sessions']

dis={}
disIT={}
for session in sessionList:
    animal=session[:6]
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    dis[animal]  =(run_distance(data))
    disIT[animal]=intertrial_displacement(data)


In [ ]:
EffectData=np.empty((4,2))
for i,session in enumerate(sessionList):
    animal=session[:6]
    EffectData[i,:]=( np.sum(disIT[animal][0:5]), np.sum(disIT[animal][30:35]) )
#     EffectData: natural locomotion, locomotion after fast speed trial

In [ ]:
EffectData/2 #division due to length of intertirla being 60 instead of 30

In [ ]:
animal='Rat369'
session='Rat369_2019_10_03_15_42'
data=Data(root,session[:6],session,param=param,redoPreprocess=True, saveAsPickle=True);


# copy the TimePaper data

In [ ]:
animalList=[
    'Rat077', 'Rat078', 'Rat084', 'Rat085', 'Rat088', 'Rat091', 'Rat095', 'Rat096', 'Rat098', 'Rat103',
    'Rat104', 'Rat106', 'Rat110', 'Rat113', 'Rat120', 'Rat121', 'Rat122', 'Rat123', 'Rat124', 'Rat131',
    'Rat132', 'Rat141', 'Rat142', 'Rat143', 'Rat144', 'Rat155', 'Rat156', 'Rat157', 'Rat158', 'Rat161',
    'Rat162', 'Rat163', 'Rat164', 'Rat165', 'Rat166', 'Rat215', 'Rat217', 'Rat218', 'Rat219', 'Rat220',
    'Rat221', 'Rat222', 'Rat223', 'Rat224', 'Rat225', 'Rat226', 'Rat227', 'Rat228', 'Rat229', 'Rat230',
    'Rat231', 'Rat232', 'Rat246', 'Rat247', 'Rat248', 'Rat249', 'Rat250', 'Rat251', 'Rat252', 'Rat253',
    'Rat254', 'Rat255', 'Rat256', 'Rat257', 'Rat258', 'Rat259', 'Rat260', 'Rat261', 'Rat262', 'Rat263',
    'Rat264', 'Rat265', 'Rat280', 'Rat281', 'Rat282', 'Rat283', 'Rat284', 'Rat285', 'Rat286', 'Rat291',
    'Rat297', 'Rat298', 'Rat299', 'Rat300', 'Rat301', 'Rat302', 'Rat303', 'Rat304', 'Rat305', 'Rat306',
    'Rat307', 'Rat308', 'Rat334', 'Rat335', 'Rat336', 'Rat338', 'Rat339', 'Rat340', 'Rat342', 'Rat344',
    'Rat346', 'Rat348', 'Rat349', 'Rat350', 'Rat352', 'Rat356', 'Rat357', 'Rat358', 'Rat359', 'Rat360',
    'Rat361'
    ]


In [ ]:
if "__file__" not in dir():
#     animalList=["Rat078"]
    PathToSourceDataFolder="/NAS02/"
    PathToDestinationDataFolder='/data/TimePaper2019/'
    RsyncArgument="rsync -auvrR --max-size=50M "
    IncludeArgument="--exclude Histology --include '*/' --include '*.tag' --include '*/*.position' --include '*/*.mat' --include '*/*.entrancetimes' --include '*/*.lickbreaktime' --include '*/*.behav_param' --exclude '*/*.coder' --exclude '*/*.seq' --exclude '*/*.avi' --exclude '*.x*' "
    CWD=os.getcwd()                                    
    os.chdir(PathToSourceDataFolder)
    for animal in sorted(animalList):
        print(animal+" in progress...")
        os.system(RsyncArgument +  IncludeArgument + animal + " " + PathToDestinationDataFolder)
        #sys.stdout.flush()
    clear_output()             
    os.chdir(CWD)
    print("Done:",animalList)

In [ ]:
profile={'Type':'Good',
         'rewardType':'Progressive',
#          'initialSpeed':'10',
#          'Speed':['10'],
         'Tag':'Reverse'
         }

animalList=batch_get_animal_list(root,profile)
print(animalList)

In [ ]:
session='Rat217_2018_04_23_11_42'
data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);


In [ ]:
i=20
plt.plot(get_speed_treadmillON(data,i)[1],get_speed_treadmillON(data,i)[0],'r.')
plt.plot(data.timeTreadmill[i],data.position[i],'g')
plt.axhline(10)
plt.axhline(-10)
# plt.plot(data.timeBin,data.speedSmoothBin[i],'r')

In [ ]:
speed,time=get_speed_treadmillON(data,i)

In [ ]:
time[np.where(np.abs(speed)>10)[0][0]]

In [ ]:
session='Rat264_2018_07_13_14_39'
data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);


In [ ]:
seqTrials,_=sequential_trials(data)
len(seqTrials)/data.nTrial

In [ ]:
np.mean([min(data.speedSmoothBin[i]) for i in seqTrials])

In [ ]:
def waiting_time(data, xFront=30, speedTolerance=10):
    detect_trial_end(data,trials=data.position.keys())
    cs=int(data.cameraSamplingRate)
    trialStart=int(cs*data.cameraToTreadmillDelay)
    
    waitTrial=[]
    waitTime=[]
    for trial,pos in data.position.items():
        
        #starting from the front
        if pos[trialStart]>xFront:
            continue
        #getting the instantaneous speed
        speed,time=get_speed_treadmillON(data,trial)
        
        if len(np.where(np.abs(speed)>abs(speedTolerance))[0]) > 0: #first time speed is faster than speedTolerance
            waitTime.append(time[np.where(np.abs(speed)>abs(speedTolerance))[0][0]])
            waitTrial.append(trial)
        
        else: #always running at the speed of treadmill, so wait=0
            waitTime.append(0)
            waitTrial.append(trial)
            
    return np.array(waitTime),np.array(waitTrial)

In [ ]:
waiting_time(data)

sequential trials

In [ ]:
def sequential_trials(data, xFront=30, xBackMin=50):
    detect_trial_end(data,trials=data.position.keys())
    cs=int(data.cameraSamplingRate)
    beamOff=scipy.stats.mode(read_file(data,paramName="consider beam state after (s)",valueType=float
                                      )).mode[0]
    beamStart=int(cs*beamOff)
    trialStart=int(cs*data.cameraToTreadmillDelay)

    seqTrial=[]
    fireTime=[]
    for trial,pos in data.position.items():
        #starting from the front
        if pos[trialStart]>xFront:
            continue
        
        #not completing the trial
        if np.isnan(data.entranceTime[trial]) or data.entranceTime[trial]==data.maxTrialDuration[trial]:
            continue

        trialPos=pos[trialStart:trialStart+int(cs*data.entranceTime[trial])]
        
        #moving a sufficiently backward!
        if trialPos.max() < xBackMin:
            continue
        
        #moving backward uninterrupted
        backwardEpoch=trialPos[beamStart:trialPos.argmax()]
        if np.any(np.diff(backwardEpoch)<0):
            continue

        #moving forward uninterrupted
        forwardEpoch=trialPos[trialPos.argmax():]
        if np.any(np.diff(forwardEpoch)>0):
            continue

        #The trial IS indeed sequential
        seqTrial.append(trial)
        fireTime.append((trialPos.argmax()/cs))

    return np.array(seqTrial),np.array(fireTime)

In [ ]:
session='Rat265_2018_07_13_14_39'
data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
a,b=sequential_trials(data, xFront=30, xBackMin=50)

In [ ]:
plt.close('all')

for trial in data.position:
    if trial in a:
        c='r'
    else: continue
    ax=plt.axes()
    ax.cla()
    ax.plot(data.position[trial],color=c)
    ax.axvline((data.entranceTime[trial]+1)*25)
    plt.show()    
    plt.draw()
    print(trial)
    breakpoint()
    clear_output()

In [ ]:
ax=plt.axes()
for trial in data.position:
    if trial in a:
        c='r'
    else: c=[0,0,0,0]
    ax.plot(data.position[trial],color=c,alpha=.1)

In [ ]:
len(a)/data.nTrial

In [ ]:
plt.hist(b,30)

In [ ]:
np.mean(b),np.median(b)

forward speed

In [ ]:
class forwardRunningSpeed:
    '''
    This class detects the running forward portion of the 
    trials and returns the forward speed
    
    '''
    def __init__(self,data,minXBack=60,minXFront=40):
        assert isinstance(data,Data), 'input data is not an instance of Data class'
        
        data.position_correction()
        self.cs          =data.cameraSamplingRate
        self.posDict     =data.position
        self.goodTrials  =data.goodTrials
        self.treadmillSpd=data.treadmillSpeed
        self.et          =data.entranceTime
        self.timeEndTrial=detect_trial_end(data)
        self.cameraDelay =data.cameraToTreadmillDelay
        self.xBack       =minXBack
        self.xFront      =minXFront
        self.session     =data.sessionPath
        self.forwardSpeed=None
    
    def compute(self):
        forwardSpeed={}
        for trial,pos in self.posDict.items():
            back_P = (pos >self.xBack).astype(int)
            _backCross=back_P-np.concatenate((np.array([False]),back_P[:-1]))
            backCrosses=np.where(_backCross==-1)[0]
            if len(backCrosses)==0:
                continue
            else:
                backCross=backCrosses[0]

            front_P= (pos >self.xFront).astype(int)
            _frontCross=front_P-np.concatenate((np.array([False]),front_P[:-1]))
            frontCrosses=np.where(_frontCross==-1)[0]
            #detect first front cross after a back cross
            frontCross=frontCrosses[frontCrosses>backCross]
            if len(frontCross)==0:
                continue
            else:
                frontCross=frontCross[0]
                            
            if frontCross/self.cs > self.timeEndTrial[trial]+self.cameraDelay:
                # treadmill had already stopped, so discard
                continue
            
            forwardSpeed[trial]=(
                                 abs(
                                     (pos[backCross]-pos[frontCross])/(frontCross-backCross)
                                    )*self.cs + self.treadmillSpd[trial]
                                 )
        
        self.forwardSpeed=forwardSpeed

        return self.forwardSpeed
                
    def conditions(self, goodOnly=False, last_n=False, sharpOnly=False):
        """
        Turn any of the input arguments, in order
        to apply that condition. Applying conditions must precede 
        running the compute() method.
        """
        assert self.forwardSpeed is None, "conditions() must be run before compute()"
        
        if goodOnly:
            self._good_only()
        if last_n:
            self._last_n(last_n)
        if sharpOnly:
            self._sharp_only(sharpOnly)
        
    def _good_only(self):
        self.posDict={trial:pos 
                      for trial,pos in self.posDict.items() 
                      if trial in self.goodTrials}
        
    def _last_n(self,last_n):
        if not last_n or not isinstance(last_n,int):
            return
        nIndex=len(self.posDict)-last_n
        self.posDict={trial:pos 
                      for trial,pos in self.posDict.items() 
                      if trial >= nIndex}
    
    def _sharp_only(self,sharpOnly: tuple):
        """
        sharpOnly: a tuple (a,b) where only trials with  a<ET<b are considered
        """
        if not sharpOnly or not isinstance(sharpOnly,tuple):
            return
        self.posDict={trial:pos 
                      for trial,pos in self.posDict.items() 
                      if self.et[trial] >= sharpOnly[0] and self.et[trial] <=sharpOnly[1]}
        

In [ ]:
session='Rat327_2019_02_28_09_14'
data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);

In [ ]:
t=forwardRunningSpeed(data)

spd=t.compute()

In [ ]:
spd

In [ ]:
session='Rat313_2018_11_27_13_49'
data=Data('/data/Teresa_MS_StriatalLesion/',session[:6],session,param={},redoPreprocess=True, saveAsPickle=False);


In [ ]:
data.position.keys()

linear regression

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3


In [ ]:
reg = LinearRegression().fit(X, y)

In [ ]:
reg.coef_

lasso

In [ ]:
from sklearn import linear_model
reg = linear_model.Lasso(alpha=0.1)
reg.fit(X,y)

In [ ]:
reg.coef_

linear regression on polynomial regressors

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly = PolynomialFeatures(degree=2)
X_new= poly.fit_transform(X)
y = np.dot(X, np.array([1, 2])) + 3


In [ ]:
reg = LinearRegression().fit(X_new, y)

In [ ]:
reg.coef_

In [ ]:
X_new

In [ ]:
sys.path.append('/home/david/MasoudRepo/masoud_treadmilltask')

In [ ]:
import DataAcquisition

In [ ]:
DataAcquisition.Animal_Tags.clear_output()

In [ ]:
class test:
    b=0
    def __init__(self):
        __class__.b=4
    @classmethod
    def met1(cls,a):
        print('class var:',a,cls)
        cls.a=a
        print(cls.__dict__)
    

In [ ]:
a=test()

a.met1(12)

b=test()

In [ ]:
class tst(dict):
    b=2
    def __init__(self):
        self.a = 1 + self.b
A=tst()
with open('/home/david/tst.p','wb') as f:
    pickle.dump(A,f)

In [ ]:
seed=[2018,4,9]
np.random.seed(seed=seed)
names=['David','Teresa','Anass','Ayse','Mostafa','Ingrid','Maryam']

np.random.shuffle(names)
print(names)

In [ ]:
session='Rat242_2018_06_25_11_29'
data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
data.position_correction();

In [ ]:
data.entranceTime[i]+data.cameraToTreadmillDelay+data.interTrialDuration[i], data.rawTime[50][-1]

In [ ]:
data.timeTreadmill[i][-1]

In [ ]:
i=50
ax=plt.figure().add_subplot(111)
ax.plot(data.rawTime[i],data.position[i],lw=3)
ax.plot(data.rawTime[i+1]+data.rawTime[i][-1],data.position[i+1],lw=3)
# ax.vlines(data.entranceTime[i]+1,10,80)

In [ ]:
profile1={'Type':'Good',
         'rewardType':'Progressive',
         'initialSpeed':'10',
         'Speed':['10'],
         'Tag':'Early-Lesion_DMS'
         }

sessions=batch_get_session_list(root,animalList=['Rat101'],profile=profile1)['Sessions']
a=[]
for session in sessions:
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    a.append(np.sum((data.entranceTime-7)**2)/len(data.entranceTime))
plt.plot(a)

In [ ]:
a=np.array([1,2,3,4,5,6])

In [ ]:
a.m

In [ ]:
filename='/data/Rat172/Experiments/Rat172_2018_02_25_14_36/Rat172_2018_02_25_14_36.raw.kwd'

with tb.open_file(filename,"r") as kwd:
    print("\n\nopenning %s"%(filename))
    dataLen,nchannels=kwd.get_node("/recordings/0/data").read().shape
    sampling_rate=kwd.get_node("/recordings/0/")._v_attrs.__getitem__('sample_rate')
print(dataLen,nchannels,sampling_rate)

In [ ]:
f = h5py.File(filename, 'r')

data = {}
data['info'] = {Rec: f['recordings'][Rec].attrs 
                for Rec in f['recordings'].keys()}

R = list(f['recordings'])[0]
if 'channel_bit_volts' in f['recordings'][R]\
                           ['application_data'].keys():
    data['channel_bit_volts'] = {Rec: f['recordings'][Rec]\
                                       ['application_data']\
                                       ['channel_bit_volts']
                                 for Rec in f['recordings'].keys()}
data['info']['0']['sample_rate']

In [ ]:
filename="/data/RatTST/Experiments/RatTST_2018_02_25_14_36/RatTST_2018_02_25_14_36.kwik"
f= h5py.File(filename, 'r')

In [ ]:
f['channel_groups']['0']['spikes']['time_samples'][:100]

In [ ]:
f['channel_groups']['0']['spikes']['time_samples'].shape

In [ ]:
import os
import h5py
import tables as tb
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

filetype='dat'
#inputFile="/home/david/TEMP/TestData/testdata openephys/2016-11-04_11-49-24/experiment1_103.raw.kwd"
#inputFile="/data/SWI002/2/SWI002_2016_09_20_12_31/SWI002_2016_09_20_12_31.raw.kwd"

#sampling_rate=kwd['recordings']['0'].attrs['sample_rate']
#f = h5py.File('inputFile','r')
fnameout = inputFile[:-7] + filetype

with tb.open_file(inputFile,"r") as f:
    #data = f['recordings'][str(0)]['data'][:,:]
    print((f.get_node("/recordings/0/data").read().shape))
    print((f.get_node('/recordings/0/')._v_attrs.__getitem__('sample_rate')))
    print(f.root.recordings._v_nchildren)

In [ ]:
f = h5py.File("/data/Rat105/Experiments/Rat105_2016_12_12_16_11/Rat105_2016_12_12_16_11.kwik",'r')
info=f['recordings']['1']['data'].shape
print((info))

In [ ]:
import KWIKphy
from KWIKphy.session import Session
f = "/data/Rat105/Experiments/Rat105_2016_12_12_16_11/Rat105_2016_12_12_16_11.kwik"
session= Session(f)

In [ ]:
print (session.model._metadata)
for shank in session.model._channel_groups:
    session.model._channel_group=shank
    print(shank,session.model._channel_group)

In [ ]:
session.model.__dict__.keys()

In [ ]:
#To read a kwik file using KWIKphy
from KWIKphy.session import Session
inputFile="/data/SWI002/2/SWI002_2016_09_20_12_52/SWI002_2016_09_20_12_52.kwik"
session= Session(inputFile)


In [ ]:
prmFile='/data/Rat107/Experiments/Rat107_2017_03_20_10_38/Rat107_2017_03_20_10_38.prm'
def a():
    x=1
    def b():
        CWD=os.getcwd()
        os.chdir('/data/Rat107/Experiments/Rat107_2017_03_20_10_38')
        prmName=os.path.basename(prmFile)
        %run -i $prmName
        os.chdir(CWD)
        return locals()
    return b()


In [ ]:
data=pd.read_csv("/data/Rat082/Rat082.tag",delim_whitespace=True,skiprows=range(5))
data

In [ ]:
a=[1,2,3,4,5,6]
try:
    a.remove('3')
except:pass
a

In [ ]:
profile1={'Type':'Good',
         #'rewardType':'Progressive',
         #'initialSpeed':'10',
         #'Speed':['10'],
         'Tag':'Early-Lesion_M1-Late-Lesion_DLS'
         }


animalList=batch_get_animal_list(root,profile1)
sessionProfile=batch_get_session_list('/data',animalList,profile1)
print (animalList)
#print(pd.DataFrame(sessionProfile))

In [ ]:
picklePath='/data/Rat084/Experiments/Rat084_2016_10_06_11_28/Analysis/preprocesseddata_binsize250ms_.p'
with open(picklePath,'rb') as f:
    data=pickle.load(f)
data.keys()

In [ ]:
import math
(math.factorial(23)/(math.factorial(9)*math.factorial(14)))

In [ ]:
profile={'Type':'Good',
         'rewardType':'Progressive',
         'initialSpeed':'10',
         'Speed':'10',
         'Tag':['Early-Lesion_DLS-BackTo10','Early-Lesion_DLS-Late-var']
        }

a=batch_get_animal_list(root,profile)
a

In [ ]:
a=[[]]*3
a

In [ ]:
x=np.arange(0,10,0.01)
y=np.sin(x)
f=plt.figure()
ax=plt.subplot(1,1,1)
plt.plot(x,y)
ax.set_xticks(range(2,8))
ax.set_xticklabels(['a','b','c','d','e','f'])
plt.show()

In [ ]:
[ax.get_xticklabels()[i].get_text() for i in range(6)]

In [ ]:
def f1 (a):
    print(a)
    return 1
def f2 (a,b):
    c=a(b)
    print(c)
    return c
f2(f1,5)

In [ ]:
def f(a,b=1):
    print(type(b))
    print(b)
f(a=1,b=np.ones(5))

In [ ]:
import inspect
inspect.getargspec(f)

In [ ]:
inspect.getargspec(update_animal_table_file)[0]

In [ ]:
dic={'a':1,'b':[]}
f(**dic)

In [ ]:
import time
print(time.perf_counter())

In [ ]:
import pims

In [ ]:
a=pims.NorpixSeq("/data/Rat106/Experiments/Rat106_2017_03_31_10_56/Rat106_2017_03_31_10_56.seq").header_dict['compression_format']